In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import joblib
import datetime

# File path
file_path = 'final_featured_stock_data_cleaned.csv'

# Save directory
save_dir = '/Users/admin/Downloads'
os.makedirs(save_dir, exist_ok=True)

# Load data
print("Loading data...")
df = pd.read_csv(file_path)

# Check basic information
print(f"Total rows: {len(df)}")
print(f"Unique stocks: {df['Stock_ID'].nunique()}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Features available: {df.columns.tolist()}")

# Data Preprocessing
# ==================

# Convert date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Handle missing values
print("Handling missing values...")
for col in df.columns:
    if df[col].isna().sum() > 0:
        if col in ['SMA_10', 'EMA_10', 'RSI_14', 'MACD', 'Signal_Line', 'OBV', 'Rolling_Mean_20', 'Rolling_Std_20', 'Bollinger_Upper', 'Bollinger_Lower']:
            # For technical indicators, forward fill is appropriate
            df[col] = df.groupby('Stock_ID')[col].fillna(method='ffill')
            # For any remaining NaNs at the beginning, backward fill
            df[col] = df.groupby('Stock_ID')[col].fillna(method='bfill')
        else:
            # For lag and rolling features
            df[col] = df.groupby('Stock_ID')[col].fillna(method='ffill')
            df[col] = df.groupby('Stock_ID')[col].fillna(method='bfill')

# Check remaining missing values
missing_values = df.isna().sum()
if missing_values.sum() > 0:
    print(f"Remaining missing values:\n{missing_values[missing_values > 0]}")
else:
    print("No missing values remain.")

# Remove Stock N which has limited data
print("Removing Stock N which has limited data...")
df = df[df['Stock_ID'] != 'Stock N']

# Select feature columns and target
# Exclude non-numeric and identifier columns
exclude_cols = ['Date', 'Stock_ID', 'Year', 'Month', 'Day', 'Day_of_Week', 'Quarter', 'time_idx']
feature_cols = [col for col in df.columns if col not in exclude_cols and col != 'Close']
target_col = 'Close'

print(f"Using {len(feature_cols)} features: {feature_cols}")
print(f"Target column: {target_col}")

# Sequence parameters
sequence_length = 30  # 30 days of history
prediction_days = 5   # Predict 5 days ahead

# Function to prepare data for LSTM for all stocks
def prepare_all_stocks_data(dataframe, feature_cols, target_col, sequence_length, prediction_days):
    """
    Prepare data for LSTM model across all stocks
    
    Parameters:
    dataframe (DataFrame): DataFrame containing all stock data
    feature_cols (list): List of feature column names
    target_col (str): Target column name
    sequence_length (int): Number of past time steps to use
    prediction_days (int): Number of days to predict ahead
    
    Returns:
    tuple: (X_train, y_train, X_val, y_val, X_test, y_test, scalers, test_stocks_data)
    """
    # Initialize containers
    X_train_all, y_train_all = [], []
    X_val_all, y_val_all = [], []
    X_test_all, y_test_all = [], []
    
    # Create global scaler for features and target
    feature_scaler = MinMaxScaler(feature_range=(0, 1))
    target_scaler = MinMaxScaler(feature_range=(0, 1))
    
    # First fit the scalers on all data
    feature_scaler.fit(dataframe[feature_cols])
    target_scaler.fit(dataframe[[target_col]])
    
    # Dictionary to store test data for each stock
    test_stocks_data = {}
    
    # Process each stock
    for stock_id in dataframe['Stock_ID'].unique():
        print(f"Processing {stock_id}...")
        stock_data = dataframe[dataframe['Stock_ID'] == stock_id].sort_values('Date').reset_index(drop=True)
        
        # Check if we have enough data
        if len(stock_data) <= sequence_length + prediction_days:
            print(f"Skipping {stock_id} - not enough data")
            continue
        
        # Transform features and target
        feature_data = feature_scaler.transform(stock_data[feature_cols])
        target_data = target_scaler.transform(stock_data[[target_col]])
        
        # Create sequences
        X, y = [], []
        for i in range(len(feature_data) - sequence_length - prediction_days + 1):
            X.append(feature_data[i:(i + sequence_length)])
            y.append(target_data[i + sequence_length + prediction_days - 1])
        
        X, y = np.array(X), np.array(y)
        
        # Split into train (70%), validation (15%), and test (15%)
        train_size = int(len(X) * 0.7)
        val_size = int(len(X) * 0.15)
        
        X_train, y_train = X[:train_size], y[:train_size]
        X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
        X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]
        
        # Store test dates and actual values for this stock
        test_stocks_data[stock_id] = {
            'test_dates': stock_data['Date'].iloc[train_size+val_size+sequence_length+prediction_days-1:].reset_index(drop=True),
            'test_actual': stock_data[target_col].iloc[train_size+val_size+sequence_length+prediction_days-1:].reset_index(drop=True),
            'X_test': X_test,
            'y_test': y_test
        }
        
        # Append to combined arrays
        X_train_all.extend(X_train)
        y_train_all.extend(y_train)
        X_val_all.extend(X_val)
        y_val_all.extend(y_val)
        X_test_all.extend(X_test)
        y_test_all.extend(y_test)
    
    # Convert to numpy arrays
    X_train_all = np.array(X_train_all)
    y_train_all = np.array(y_train_all)
    X_val_all = np.array(X_val_all)
    y_val_all = np.array(y_val_all)
    X_test_all = np.array(X_test_all)
    y_test_all = np.array(y_test_all)
    
    print(f"Combined training shape: {X_train_all.shape}, {y_train_all.shape}")
    print(f"Combined validation shape: {X_val_all.shape}, {y_val_all.shape}")
    print(f"Combined testing shape: {X_test_all.shape}, {y_test_all.shape}")
    
    return (X_train_all, y_train_all, X_val_all, y_val_all, X_test_all, y_test_all,
            (feature_scaler, target_scaler), test_stocks_data)

# Function to build and train LSTM model
def build_lstm_model(X_train, y_train, X_val, y_val, save_path):
    """
    Build and train LSTM model
    
    Parameters:
    X_train (array): Training features
    y_train (array): Training target
    X_val (array): Validation features
    y_val (array): Validation target
    save_path (str): Path to save the model
    
    Returns:
    model: Trained LSTM model
    history: Training history
    """
    # Define model architecture
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    
    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True
    )
    
    # Model checkpoint
    checkpoint = ModelCheckpoint(
        filepath=save_path,
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, checkpoint],
        verbose=1
    )
    
    return model, history

# Function to evaluate model performance by stock
def evaluate_model_by_stock(model, test_stocks_data, target_scaler, save_dir):
    """
    Evaluate model performance for each stock
    
    Parameters:
    model: Trained LSTM model
    test_stocks_data (dict): Dictionary containing test data for each stock
    target_scaler: Scaler used for target variable
    save_dir (str): Directory to save output
    
    Returns:
    dict: Performance metrics for each stock
    """
    results = {}
    
    # Create plot directory
    plots_dir = os.path.join(save_dir, 'plots')
    os.makedirs(plots_dir, exist_ok=True)
    
    # Overall metrics containers
    all_actual = []
    all_pred = []
    
    for stock_id, stock_data in test_stocks_data.items():
        print(f"Evaluating {stock_id}...")
        X_test = stock_data['X_test']
        y_test = stock_data['y_test']
        test_dates = stock_data['test_dates']
        
        if len(X_test) == 0:
            print(f"No test data for {stock_id}, skipping...")
            continue
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Inverse transform predictions and actual values
        y_test_inv = target_scaler.inverse_transform(y_test)
        y_pred_inv = target_scaler.inverse_transform(y_pred)
        
        # Accumulate for overall metrics
        all_actual.extend(y_test_inv.flatten())
        all_pred.extend(y_pred_inv.flatten())
        
        # Calculate metrics
        mse = mean_squared_error(y_test_inv, y_pred_inv)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test_inv, y_pred_inv)
        r2 = r2_score(y_test_inv, y_pred_inv)
        
        # Calculate MAPE
        mape = np.mean(np.abs((y_test_inv - y_pred_inv) / y_test_inv)) * 100
        
        print(f"\nPerformance metrics for {stock_id}:")
        print(f"MSE: {mse:.2f}")
        print(f"RMSE: {rmse:.2f}")
        print(f"MAE: {mae:.2f}")
        print(f"MAPE: {mape:.2f}%")
        print(f"R^2: {r2:.2f}")
        
        # Plot actual vs predicted
        plt.figure(figsize=(14, 7))
        plt.plot(y_test_inv, label='Actual')
        plt.plot(y_pred_inv, label='Predicted')
        plt.title(f'{stock_id} - Actual vs Predicted ({prediction_days} days ahead)')
        plt.xlabel('Trading Days')
        plt.ylabel('Price')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        
        plt.savefig(os.path.join(plots_dir, f'{stock_id}_prediction.png'))
        plt.close()
        
        results[stock_id] = {
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'mape': mape,
            'r2': r2
        }
    
    # Calculate overall metrics
    overall_mse = mean_squared_error(all_actual, all_pred)
    overall_rmse = np.sqrt(overall_mse)
    overall_mae = mean_absolute_error(all_actual, all_pred)
    overall_r2 = r2_score(all_actual, all_pred)
    
    # Calculate overall MAPE
    overall_mape = np.mean(np.abs((np.array(all_actual) - np.array(all_pred)) / np.array(all_actual))) * 100
    
    print("\nOverall Performance Metrics:")
    print(f"MSE: {overall_mse:.2f}")
    print(f"RMSE: {overall_rmse:.2f}")
    print(f"MAE: {overall_mae:.2f}")
    print(f"MAPE: {overall_mape:.2f}%")
    print(f"R^2: {overall_r2:.2f}")
    
    results['overall'] = {
        'mse': overall_mse,
        'rmse': overall_rmse,
        'mae': overall_mae,
        'mape': overall_mape,
        'r2': overall_r2
    }
    
    return results

# Function to make future predictions for all stocks
def predict_future(model, df, feature_cols, target_col, feature_scaler, target_scaler, 
                   sequence_length, prediction_days, num_future_days, save_dir):
    """
    Make future predictions for all stocks
    
    Parameters:
    model: Trained LSTM model
    df (DataFrame): Original dataframe
    feature_cols (list): List of feature column names
    target_col (str): Target column name
    feature_scaler: Scaler used for features
    target_scaler: Scaler used for target
    sequence_length (int): Length of input sequence
    prediction_days (int): Days ahead the model was trained to predict
    num_future_days (int): Number of days to predict into future (multiple of prediction_days)
    save_dir (str): Directory to save output
    
    Returns:
    dict: Dictionary with future predictions for each stock
    """
    future_predictions = {}
    predictions_dir = os.path.join(save_dir, 'future_predictions')
    os.makedirs(predictions_dir, exist_ok=True)
    
    # Ensure num_future_days is a multiple of prediction_days
    num_steps = num_future_days // prediction_days
    if num_future_days % prediction_days != 0:
        num_steps += 1
        print(f"Adjusting to {num_steps * prediction_days} future days (multiple of {prediction_days})")
    
    for stock_id in df['Stock_ID'].unique():
        if stock_id == 'Stock N':
            continue
            
        print(f"Generating future predictions for {stock_id}...")
        stock_data = df[df['Stock_ID'] == stock_id].sort_values('Date').reset_index(drop=True)
        
        # Get the most recent date
        last_date = stock_data['Date'].iloc[-1]
        print(f"Last available date: {last_date}")
        
        # Get the last sequence
        last_sequence = stock_data.iloc[-sequence_length:][feature_cols]
        last_sequence_scaled = feature_scaler.transform(last_sequence)
        last_sequence_scaled = np.array([last_sequence_scaled])
        
        # Prepare containers for predictions
        future_dates = []
        predictions = []
        
        # Generate trading dates (excluding weekends)
        current_date = last_date
        days_added = 0
        
        while days_added < num_steps * prediction_days:
            current_date += pd.Timedelta(days=1)
            # Skip weekends (0 = Monday, 6 = Sunday)
            if current_date.weekday() < 5:
                days_added += 1
                future_dates.append(current_date)
        
        # Make future predictions
        current_sequence = last_sequence_scaled.copy()
        
        for step in range(num_steps):
            # Predict next value
            next_pred = model.predict(current_sequence)
            
            # Store predictions for this step
            for i in range(prediction_days):
                if step * prediction_days + i < len(future_dates):
                    # Convert prediction back to original scale
                    pred_orig = target_scaler.inverse_transform(next_pred)[0, 0]
                    predictions.append(pred_orig)
            
            # Update sequence for next prediction
            # In a real implementation, we would need to update all features
            # Here we're making a simplified approach by reusing the last sequence
            # and updating only the target values
            # For a more accurate approach, you would need to recalculate technical indicators
            
            # Move the sequence window forward
            current_sequence = np.roll(current_sequence, -prediction_days, axis=1)
            
            # Update the last values with the new prediction
            # This is a simplification - in practice, you would update all features
            last_idx = current_sequence.shape[1] - 1
            current_sequence[0, last_idx, 0] = next_pred[0, 0]  # Assuming target is first feature
        
        # Trim predictions to match requested future days
        predictions = predictions[:num_future_days]
        future_dates = future_dates[:num_future_days]
        
        # Create DataFrame with predictions
        predictions_df = pd.DataFrame({
            'Date': future_dates,
            'Predicted_Price': predictions
        })
        
        # Save predictions
        predictions_df.to_csv(os.path.join(predictions_dir, f'{stock_id}_future_predictions.csv'), index=False)
        
        # Plot historical prices and predictions
        plt.figure(figsize=(14, 7))
        
        # Historical data
        historical_dates = stock_data['Date'].iloc[-100:]  # Last 100 days
        historical_prices = stock_data[target_col].iloc[-100:]
        
        plt.plot(historical_dates, historical_prices, label='Historical', color='blue')
        plt.plot(future_dates, predictions, label='Predicted', color='red', linestyle='--')
        
        plt.title(f'{stock_id} - Future Price Predictions ({num_future_days} days)')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        plt.savefig(os.path.join(predictions_dir, f'{stock_id}_future_predictions.png'))
        plt.close()
        
        future_predictions[stock_id] = predictions_df
    
    return future_predictions

# Main execution
if __name__ == "__main__":
    print("Starting LSTM model training for stock price prediction...")
    
    # Timestamp for model and outputs
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    model_path = os.path.join(save_dir, f'lstm_stock_model_{timestamp}.h5')
    
    # Prepare data for all stocks
    (X_train, y_train, X_val, y_val, X_test, y_test, 
     scalers, test_stocks_data) = prepare_all_stocks_data(
        df, feature_cols, target_col, sequence_length, prediction_days)
    
    feature_scaler, target_scaler = scalers
    
    # Save scalers
    scaler_path = os.path.join(save_dir, f'scalers_{timestamp}.pkl')
    joblib.dump(scalers, scaler_path)
    print(f"Scalers saved to: {scaler_path}")
    
    # Build and train model
    print("Training LSTM model...")
    model, history = build_lstm_model(X_train, y_train, X_val, y_val, model_path)
    
    # Save training history
    history_df = pd.DataFrame(history.history)
    history_path = os.path.join(save_dir, f'training_history_{timestamp}.csv')
    history_df.to_csv(history_path, index=False)
    print(f"Training history saved to: {history_path}")
    
    # Plot training history
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 1, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    history_plot_path = os.path.join(save_dir, f'training_loss_{timestamp}.png')
    plt.savefig(history_plot_path)
    plt.close()
    print(f"Training loss plot saved to: {history_plot_path}")
    
    # Evaluate model on test data for each stock
    print("Evaluating model performance...")
    evaluation_results = evaluate_model_by_stock(model, test_stocks_data, target_scaler, save_dir)
    
    # Save evaluation results
    results_df = pd.DataFrame.from_dict({k: v for k, v in evaluation_results.items() 
                                        if k != 'overall'}, orient='index')
    results_path = os.path.join(save_dir, f'evaluation_results_{timestamp}.csv')
    results_df.to_csv(results_path)
    print(f"Evaluation results saved to: {results_path}")
    
    # Generate future predictions for 2023-2024 (approximately 500 trading days)
    print("Generating future predictions...")
    future_days = 500
    future_predictions = predict_future(
        model, df, feature_cols, target_col, feature_scaler, target_scaler,
        sequence_length, prediction_days, future_days, save_dir
    )
    
    # Save model parameters info
    params = {
        'sequence_length': sequence_length,
        'prediction_days': prediction_days,
        'feature_columns': feature_cols,
        'target_column': target_col,
        'model_path': model_path,
        'scaler_path': scaler_path,
        'training_size': len(X_train),
        'validation_size': len(X_val),
        'testing_size': len(X_test)
    }
    
    params_df = pd.DataFrame([params])
    params_path = os.path.join(save_dir, f'model_parameters_{timestamp}.csv')
    params_df.to_csv(params_path, index=False)
    print(f"Model parameters saved to: {params_path}")
    
    print("\nLSTM model training and evaluation complete!")
    print(f"Model saved to: {model_path}")
    print(f"Future predictions for 2023-2024 saved to: {os.path.join(save_dir, 'future_predictions')}")

Loading data...
Total rows: 24096
Unique stocks: 20
Date range: 2018-01-02 to 2022-12-30
Features available: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Stock_ID', 'time_idx', 'SMA_10', 'EMA_10', 'RSI_14', 'Rolling_Mean_20', 'Rolling_Std_20', 'Bollinger_Upper', 'Bollinger_Lower', 'EMA_12', 'EMA_26', 'MACD', 'Signal_Line', 'OBV', 'Year', 'Month', 'Day', 'Day_of_Week', 'Quarter', 'Close_Lag_1', 'Close_Lag_3', 'Close_Lag_7', 'Close_Rolling_5', 'Close_Rolling_10', 'Close_Rolling_20']
Handling missing values...


/var/folders/50/q60d7z1n48s4vnhtd7ygxgj80000gn/T/ipykernel_72269/2303230976.py:42: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df[col] = df.groupby('Stock_ID')[col].fillna(method='ffill')
/var/folders/50/q60d7z1n48s4vnhtd7ygxgj80000gn/T/ipykernel_72269/2303230976.py:42: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df.groupby('Stock_ID')[col].fillna(method='ffill')
/var/folders/50/q60d7z1n48s4vnhtd7ygxgj80000gn/T/ipykernel_72269/2303230976.py:44: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df[col] = df.groupby('Stock_ID

No missing values remain.
Removing Stock N which has limited data...
Using 22 features: ['High', 'Low', 'Open', 'Volume', 'SMA_10', 'EMA_10', 'RSI_14', 'Rolling_Mean_20', 'Rolling_Std_20', 'Bollinger_Upper', 'Bollinger_Lower', 'EMA_12', 'EMA_26', 'MACD', 'Signal_Line', 'OBV', 'Close_Lag_1', 'Close_Lag_3', 'Close_Lag_7', 'Close_Rolling_5', 'Close_Rolling_10', 'Close_Rolling_20']
Target column: Close
Starting LSTM model training for stock price prediction...
Processing Stock A...
Processing Stock B...
Processing Stock C...
Processing Stock D...
Processing Stock E...
Processing Stock F...
Processing Stock G...
Processing Stock H...
Processing Stock I...
Processing Stock J...
Processing Stock K...
Processing Stock L...
Processing Stock M...
Processing Stock O...
Processing Stock P...
Processing Stock Q...
Processing Stock R...
Processing Stock S...
Processing Stock T...
Combined training shape: (16325, 30, 22), (16325, 1)
Combined validation shape: (3494, 30, 22), (3494, 1)
Combined testin

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100


...

511/511 ━━━━━━━━━━━━━━━━━━━━ 17s 32ms/step - loss: 4.6947e-04 - val_loss: 1.8485e-04
Epoch 19/100
510/511 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 4.4222e-04
Epoch 19: val_loss improved from 0.00018 to 0.00018, saving model to /Users/admin/Downloads/lstm_stock_model_20250327_133439.h5


511/511 ━━━━━━━━━━━━━━━━━━━━ 17s 33ms/step - loss: 4.4223e-04 - val_loss: 1.7993e-04
Epoch 20/100
511/511 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 4.3741e-04
Epoch 20: val_loss improved from 0.00018 to 0.00017, saving model to /Users/admin/Downloads/lstm_stock_model_20250327_133439.h5


511/511 ━━━━━━━━━━━━━━━━━━━━ 17s 32ms/step - loss: 4.3740e-04 - val_loss: 1.6753e-04
Epoch 21/100
511/511 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 4.1093e-04
Epoch 21: val_loss did not improve from 0.00017
511/511 ━━━━━━━━━━━━━━━━━━━━ 16s 32ms/step - loss: 4.1095e-04 - val_loss: 2.4304e-04
Epoch 22/100
511/511 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 4.3596e-04
Epoch 22: val_loss did not improve from 0.00017
511/511 ━━━━━━━━━━━━━━━━━━━━ 16s 32ms/step - loss: 4.3595e-04 - val_loss: 4.4412e-04
Epoch 23/100
510/511 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 4.5650e-04
Epoch 23: val_loss did not improve from 0.00017
511/511 ━━━━━━━━━━━━━━━━━━━━ 16s 32ms/step - loss: 4.5634e-04 - val_loss: 1.9638e-04
Epoch 24/100
511/511 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 3.9320e-04
Epoch 24: val_loss did not improve from 0.00017
511/511 ━━━━━━━━━━━━━━━━━━━━ 17s 33ms/step - loss: 3.9328e-04 - val_loss: 1.8489e-04
Epoch 25/100
511/511 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 4.2300e-04
Epoch 25: val

2025-03-27 13:44:34.969453: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step

Performance metrics for Stock A:
MSE: 47926.42
RMSE: 218.92
MAE: 171.46
MAPE: 3.88%
R^2: 0.64
Evaluating Stock B...
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

Performance metrics for Stock B:
MSE: 4700.85
RMSE: 68.56
MAE: 53.77
MAPE: 2.47%
R^2: 0.18
Evaluating Stock C...
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

Performance metrics for Stock C:
MSE: 5531.89
RMSE: 74.38
MAE: 56.55
MAPE: 2.07%
R^2: -0.15
Evaluating Stock D...
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

Performance metrics for Stock D:
MSE: 284.54
RMSE: 16.87
MAE: 12.96
MAPE: 3.43%
R^2: 0.78
Evaluating Stock E...
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

Performance metrics for Stock E:
MSE: 71594.16
RMSE: 267.57
MAE: 220.99
MAPE: 2.96%
R^2: 0.52
Evaluating Stock F...
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

Performance metrics for Stock F:
MSE: 4635.51
RMSE: 68.08
MAE: 50.58
MAPE: 3.66%
R^2: 0.82
Evaluating Stock G...
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

Performance metrics for Stock G:
MSE: 8059.